# Basic data analysis
Do not run this notebook! Radiomics extraction results were really bad and cannot be exploited!!

In [120]:
import pandas as pd 
import csv


## Correlate Patient number and File Number 

In [121]:
# get all clinics file
clinics_file_path = "/mnt/c/Users/tachenne/delta-rad/Recueil clinique pancréas MRIdian.xls"
clinics_xls = pd.ExcelFile(clinics_file_path)
print(clinics_xls.sheet_names)
#clinics_df = xls.parse

['Patients', 'Dosi Volumes cibles ', 'Dosi OAR ', 'TTT', 'TTT OAR', 'Moyennes OAR ', 'Fin ttt', '1 mois ', '3 mois ', '6 mois ', '9 mois ', '1 an', '2 ans', 'Survies ']


In [122]:
# get patient list file (only patient numbers and ID ICM)
patient_file = "/mnt/c/Users/tachenne/delta-rad/Liste patients pancréas exports.xlsx"
patients_xlsx = pd.read_excel(patient_file, index_col=0)
patients_xlsx.head()

,Initiales,ID ICM,Notes,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Liste recueil clinique,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
Patient 1,CF,1403778.0,NaN,NaN,NaN,NaN,NaN,NaN,1403778.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 2,CL,1906503.0,NaN,NaN,NaN,NaN,NaN,NaN,1906503.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 3,CR,1904130.0,NaN,NaN,NaN,NaN,NaN,NaN,1904130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 4,BK,1905976.0,NaN,NaN,NaN,NaN,NaN,NaN,1905976.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 5,LJ,1904380.0,NaN,NaN,NaN,NaN,NaN,NaN,1907649.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
patients_xlsx.index

Index([ 'Patient 1',  'Patient 2',  'Patient 3',  'Patient 4',  'Patient 5',
        'Patient 6',  'Patient 7',  'Patient 8',  'Patient 9', 'Patient 10',
       'Patient 11', 'Patient 12', 'Patient 13', 'Patient 14', 'Patient 15',
       'Patient 16', 'Patient 17', 'Patient 18', 'Patient 19', 'Patient 20',
       'Patient 21', 'Patient 22', 'Patient 23', 'Patient 24', 'Patient 25',
       'Patient 26', 'Patient 27', 'Patient 28', 'Patient 29', 'Patient 30',
       'Patient 31', 'Patient 32', 'Patient 33', 'Patient 34', 'Patient 35',
       'Patient 36', 'Patient 37', 'Patient 38', 'Patient 39', 'Patient 40',
       'Patient 41', 'Patient 42', 'Patient 43', 'Patient 44', 'Patient 45',
       'Patient 46', 'Patient 47', 'Patient 48', 'Patient 49', 'Patient 50',
       'Patient 51', 'Patient 52', 'Patient 53', 'Patient 54', 'Patient 55',
       'Patient 56', 'Patient 57', 'Patient 58', 'Patient 59', 'Patient 60',
       'Patient 61', 'Patient 62', 'Patient 63', 'Patient 64', 'Patient 65',

In [124]:
patients_df = patients_xlsx[['ID ICM']]
patients_df.head()

,ID ICM
Patient 1,1403778.0
Patient 2,1906503.0
Patient 3,1904130.0
Patient 4,1905976.0
Patient 5,1904380.0


## Check that we have all values 

In [125]:
patients_list = patients_df.index.dropna()
print("Number of patients: ", len(patients_list))
pyrad_results_folder = "/mnt/c/Users/tachenne/delta-rad/Results GTV-PTV/"
misses_fraction = []
misses_gtv_ptv_row = []
for i in range(1, len(patients_list)+1):
    #print("Patient index:", i)
    filename = pyrad_results_folder + 'extractedFeaturePat' + str(i) + '.csv'
    extracted_feat_df = pd.read_csv(filename, sep =';', header=None, names=['ID', 'Image'])
    indexes = extracted_feat_df.index.dropna() # get index 
    if len(indexes) < 13:
        print("Error: miss one fraction for patient {}".format(i))
        misses_fraction.append((i, len(indexes)))

    elif len(indexes) < 25:
        #print("Error: miss PTV or GTV specific rows for patient {}".format(i))
        misses_gtv_ptv_row.append(i)

print("Patients ID who missed a fraction:", misses_fraction)
print("Number of patients with incomplete csv:", len(misses_fraction))
print("Patients ID which csv is not well made:", misses_gtv_ptv_row)



Number of patients:  86
Error: miss one fraction for patient 54
Error: miss one fraction for patient 56
Error: miss one fraction for patient 57
Error: miss one fraction for patient 61
Error: miss one fraction for patient 62
Error: miss one fraction for patient 63
Error: miss one fraction for patient 64
Error: miss one fraction for patient 66
Error: miss one fraction for patient 70
Error: miss one fraction for patient 71
Error: miss one fraction for patient 74
Error: miss one fraction for patient 79
Error: miss one fraction for patient 80
Error: miss one fraction for patient 81
Error: miss one fraction for patient 82
Error: miss one fraction for patient 84
Error: miss one fraction for patient 85
Error: miss one fraction for patient 86
Patients ID who missed a fraction: [(54, 11), (56, 11), (57, 11), (61, 11), (62, 11), (63, 11), (64, 11), (66, 9), (70, 11), (71, 12), (74, 7), (79, 11), (80, 7), (81, 11), (82, 11), (84, 11), (85, 11), (86, 11)]
Number of patients with incomplete csv: 18


## Compute radiomics variations between fraction 1 (F1) and F2 for GTV mask

In [126]:
features_list = [
    'original_shape_Elongation', 'original_shape_Flatness', 'original_shape_LeastAxisLength',
    'original_shape_MajorAxisLength', 'original_shape_Maximum2DDiameterColumn',
    'original_shape_Maximum2DDiameterRow', 'original_shape_Maximum2DDiameterSlice',
    'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume',
    'original_shape_MinorAxisLength', 'original_shape_Sphericity',
    'original_shape_SurfaceArea', 'original_shape_SurfaceVolumeRatio',
    'original_shape_VoxelVolume', 'original_firstorder_10Percentile',
    'original_firstorder_90Percentile', 'original_firstorder_Energy',
    'original_firstorder_Entropy', 'original_firstorder_InterquartileRange',
    'original_firstorder_Kurtosis', 'original_firstorder_Maximum',
    'original_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Mean',
    'original_firstorder_Median', 'original_firstorder_Minimum',
    'original_firstorder_Range', 'original_firstorder_RobustMeanAbsoluteDeviation',
    'original_firstorder_RootMeanSquared', 'original_firstorder_Skewness',
    'original_firstorder_TotalEnergy', 'original_firstorder_Uniformity',
    'original_firstorder_Variance', 'original_glcm_Autocorrelation',
    'original_glcm_ClusterProminence', 'original_glcm_ClusterShade',
    'original_glcm_ClusterTendency', 'original_glcm_Contrast',
    'original_glcm_Correlation', 'original_glcm_DifferenceAverage',
    'original_glcm_DifferenceEntropy', 'original_glcm_DifferenceVariance',
    'original_glcm_Id', 'original_glcm_Idm', 'original_glcm_Idmn', 'original_glcm_Idn',
    'original_glcm_Imc1', 'original_glcm_Imc2', 'original_glcm_InverseVariance',
    'original_glcm_JointAverage', 'original_glcm_JointEnergy',
    'original_glcm_JointEntropy', 'original_glcm_MCC', 'original_glcm_MaximumProbability',
    'original_glcm_SumAverage', 'original_glcm_SumEntropy', 'original_glcm_SumSquares',
    'original_glrlm_GrayLevelNonUniformity', 'original_glrlm_GrayLevelNonUniformityNormalized',
    'original_glrlm_GrayLevelVariance', 'original_glrlm_HighGrayLevelRunEmphasis',
    'original_glrlm_LongRunEmphasis', 'original_glrlm_LongRunHighGrayLevelEmphasis',
    'original_glrlm_LongRunLowGrayLevelEmphasis', 'original_glrlm_LowGrayLevelRunEmphasis',
    'original_glrlm_RunEntropy', 'original_glrlm_RunLengthNonUniformity',
    'original_glrlm_RunLengthNonUniformityNormalized', 'original_glrlm_RunPercentage',
    'original_glrlm_RunVariance', 'original_glrlm_ShortRunEmphasis',
    'original_glrlm_ShortRunHighGrayLevelEmphasis', 'original_glrlm_ShortRunLowGrayLevelEmphasis',
    'original_glszm_GrayLevelNonUniformity', 'original_glszm_GrayLevelNonUniformityNormalized',
    'original_glszm_GrayLevelVariance', 'original_glszm_HighGrayLevelZoneEmphasis',
    'original_glszm_LargeAreaEmphasis', 'original_glszm_LargeAreaHighGrayLevelEmphasis',
    'original_glszm_LargeAreaLowGrayLevelEmphasis', 'original_glszm_LowGrayLevelZoneEmphasis',
    'original_glszm_SizeZoneNonUniformity', 'original_glszm_SizeZoneNonUniformityNormalized',
    'original_glszm_SmallAreaEmphasis', 'original_glszm_SmallAreaHighGrayLevelEmphasis',
    'original_glszm_SmallAreaLowGrayLevelEmphasis', 'original_glszm_ZoneEntropy',
    'original_glszm_ZonePercentage', 'original_glszm_ZoneVariance',
    'original_gldm_DependenceEntropy', 'original_gldm_DependenceNonUniformity',
    'original_gldm_DependenceNonUniformityNormalized', 'original_gldm_DependenceVariance',
    'original_gldm_GrayLevelNonUniformity', 'original_gldm_GrayLevelVariance',
    'original_gldm_HighGrayLevelEmphasis', 'original_gldm_LargeDependenceEmphasis',
    'original_gldm_LargeDependenceHighGrayLevelEmphasis',
    'original_gldm_LargeDependenceLowGrayLevelEmphasis',
    'original_gldm_LowGrayLevelEmphasis', 'original_gldm_SmallDependenceEmphasis',
    'original_gldm_SmallDependenceHighGrayLevelEmphasis',
    'original_gldm_SmallDependenceLowGrayLevelEmphasis', 'original_ngtdm_Busyness',
    'original_ngtdm_Coarseness', 'original_ngtdm_Complexity', 'original_ngtdm_Contrast',
    'original_ngtdm_Strength'
]

In [127]:
other_info = [
    "diagnostics_Versions_PyRadiomics",
    "diagnostics_Versions_Numpy",
    "diagnostics_Versions_SimpleITK",
    "diagnostics_Versions_PyWavelet",
    "diagnostics_Versions_Python",
    "diagnostics_Configuration_Settings",
    "diagnostics_Configuration_EnabledImageTypes",
    "diagnostics_Image-original_Hash",
    "diagnostics_Image-original_Dimensionality",
    "diagnostics_Image-original_Spacing",
    "diagnostics_Image-original_Size",
    "diagnostics_Image-original_Mean",
    "diagnostics_Image-original_Minimum",
    "diagnostics_Image-original_Maximum",
    "diagnostics_Mask-original_Hash",
    "diagnostics_Mask-original_Spacing",
    "diagnostics_Mask-original_Size",
    "diagnostics_Mask-original_BoundingBox",
    "diagnostics_Mask-original_VoxelNum",
    "diagnostics_Mask-original_VolumeNum",
    "diagnostics_Mask-original_CenterOfMassIndex",
    "diagnostics_Mask-original_CenterOfMass",
    "diagnostics_Image-interpolated_Spacing",
    "diagnostics_Image-interpolated_Size",
    "diagnostics_Image-interpolated_Mean",
    "diagnostics_Image-interpolated_Minimum",
    "diagnostics_Image-interpolated_Maximum",
    "diagnostics_Mask-interpolated_Spacing",
    "diagnostics_Mask-interpolated_Size",
    "diagnostics_Mask-interpolated_BoundingBox",
    "diagnostics_Mask-interpolated_VoxelNum",
    "diagnostics_Mask-interpolated_VolumeNum",
    "diagnostics_Mask-interpolated_CenterOfMassIndex",
    "diagnostics_Mask-interpolated_CenterOfMass",
    "diagnostics_Mask-interpolated_Mean",
    "diagnostics_Mask-interpolated_Minimum",
    "diagnostics_Mask-interpolated_Maximum",
    "diagnostics_Mask-resegmented_Spacing",
    "diagnostics_Mask-resegmented_Size",
    "diagnostics_Mask-resegmented_BoundingBox",
    "diagnostics_Mask-resegmented_VoxelNum",
    "diagnostics_Mask-resegmented_VolumeNum",
    "diagnostics_Mask-resegmented_CenterOfMassIndex",
    "diagnostics_Mask-resegmented_CenterOfMass",
    "diagnostics_Mask-resegmented_Mean",
    "diagnostics_Mask-resegmented_Minimum",
    "diagnostics_Mask-resegmented_Maximum"
]

In [128]:
# define dataframe columns
names = ['ID', 'Image', 'Mask'] + other_info + features_list
print(names)

['ID', 'Image', 'Mask', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass', 'diagnostics_Image-interpolated_Spacing', 'diagnostics_Image-interpolated_Size', 'diagnostics_Image-interpolated_Mean', 'diagnostics_Image-interpolate

In [129]:
# define columns of simu/F1 delta rad next dataframe 
new_feature_list = ['delta_f1_f2_' + x for x in features_list]
print(new_feature_list)

['delta_f1_f2_original_shape_Elongation', 'delta_f1_f2_original_shape_Flatness', 'delta_f1_f2_original_shape_LeastAxisLength', 'delta_f1_f2_original_shape_MajorAxisLength', 'delta_f1_f2_original_shape_Maximum2DDiameterColumn', 'delta_f1_f2_original_shape_Maximum2DDiameterRow', 'delta_f1_f2_original_shape_Maximum2DDiameterSlice', 'delta_f1_f2_original_shape_Maximum3DDiameter', 'delta_f1_f2_original_shape_MeshVolume', 'delta_f1_f2_original_shape_MinorAxisLength', 'delta_f1_f2_original_shape_Sphericity', 'delta_f1_f2_original_shape_SurfaceArea', 'delta_f1_f2_original_shape_SurfaceVolumeRatio', 'delta_f1_f2_original_shape_VoxelVolume', 'delta_f1_f2_original_firstorder_10Percentile', 'delta_f1_f2_original_firstorder_90Percentile', 'delta_f1_f2_original_firstorder_Energy', 'delta_f1_f2_original_firstorder_Entropy', 'delta_f1_f2_original_firstorder_InterquartileRange', 'delta_f1_f2_original_firstorder_Kurtosis', 'delta_f1_f2_original_firstorder_Maximum', 'delta_f1_f2_original_firstorder_MeanA

In [140]:
def get_data_from_csv(name_exam1: str, name_exam2: str, other_exam_name: str): 
    ''' Obtains delta-radiomics data for all patients of the csv between 2 specified exams.'''

    # Créer une liste vide pour stocker les données
    data = []
    error_patients = []

    # Ajouter les données pour chaque patient
    for i in range(1, len(patients_list)+1):
        # define the row to extract 
        if i < 10: 
            patient_index = '0' + str(i)
        else: 
            patient_index = str(i)

        row_to_extract1 = 'Data/Patient' + patient_index + '/mask_dir/Patient' + patient_index + name_exam1
        row_to_extract2 = 'Data/Patient' + patient_index + '/mask_dir/Patient' + patient_index + name_exam2
        # open patient csv file 
        filename = pyrad_results_folder + 'extractedFeaturePat' + str(i) + '.csv'
        extracted_feat_df = pd.read_csv(filename, sep =';', header=None, names=names)

        if i > 70: 
            print(extracted_feat_df['Mask'].values, row_to_extract1)


        row_data = []
        for feature in features_list:
            if (row_to_extract1 in extracted_feat_df['Mask'].astype(str).values) and (row_to_extract2 in extracted_feat_df['Mask'].astype(str).values):
                feature_value1 = float(extracted_feat_df.loc[extracted_feat_df['Mask'] == row_to_extract1, feature].iloc[0].replace(',', '.')) # read feature at simu gtv row 
                feature_value2 = float(extracted_feat_df.loc[extracted_feat_df['Mask'] == row_to_extract2, feature].iloc[0].replace(',', '.')) # read feature at f1 gtv row 
                delta_feature = feature_value2 - feature_value1 # compute the difference 
                row_data.append(delta_feature) # add the value to raw_data 
            #elif row_to_extract2 in extracted_feat_df['Mask'].astype(str).values: 
            #    row_to_extract1 = 'Data/Patient' + patient_index + '/mask_dir/Patient' + patient_index + other_exam_name # Sometimes row names are different
            #    if (row_to_extract1 in extracted_feat_df['Mask'].astype(str).values) and (row_to_extract2 in extracted_feat_df['Mask'].astype(str).values):
            #        feature_value1 = float(extracted_feat_df.loc[extracted_feat_df['Mask'] == row_to_extract1, feature].iloc[0].replace(',', '.')) # read feature at simu gtv row 
            #        feature_value2 = float(extracted_feat_df.loc[extracted_feat_df['Mask'] == row_to_extract2, feature].iloc[0].replace(',', '.')) # read feature at f1 gtv row 
            #        delta_feature = feature_value2 / feature_value1 # compute the quotient 
            #        row_data.append(delta_feature) # add the value to raw_data 
            #    else: 
            #        row_data.append(None)
            #        if patient_index not in error_patients: 
            #            error_patients.append(patient_index)

            else: 
                row_data.append(None)
                if patient_index not in error_patients: 
                    error_patients.append(patient_index)

        data.append(row_data)

    return data, error_patients

In [141]:
#data, error_patients = get_data_from_csv('_mridian_ttt_1_gtv,nii', '_mridian_ttt_2_gtv,nii', 'aa') 
#print("Patient indexes with errors: ", error_patients)
# Créer le DataFrame
data_bis, error_patients = get_data_from_csv('_mridian_ttt_1_gtv.nii', '_mridian_ttt_2_gtv.nii', 'aa')
#data.extend(data_bis)
print("Patient indexes with errors: ", error_patients)

delta_f1_f2_df = pd.DataFrame(data_bis, index=patients_list, columns=new_feature_list)


[nan nan nan nan nan nan nan nan nan nan nan nan] Data/Patient71/mask_dir/Patient71_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan nan nan nan nan nan nan] Data/Patient72/mask_dir/Patient72_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan nan nan nan nan nan nan] Data/Patient73/mask_dir/Patient73_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan] Data/Patient74/mask_dir/Patient74_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan nan nan nan nan nan nan] Data/Patient75/mask_dir/Patient75_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan nan nan nan nan nan nan] Data/Patient76/mask_dir/Patient76_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan nan nan nan nan nan nan] Data/Patient77/mask_dir/Patient77_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan nan nan nan nan nan nan] Data/Patient78/mask_dir/Patient78_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan nan nan nan nan] Data/Patient79/mask_dir/Patient79_mridian_ttt_1_gtv.nii
[nan nan nan nan nan nan nan] Data/Patient80/m

These patients do not have simulation!!

In [135]:
delta_f1_f2_df


,delta_f1_f2_original_shape_Elongation,delta_f1_f2_original_shape_Flatness,delta_f1_f2_original_shape_LeastAxisLength,delta_f1_f2_original_shape_MajorAxisLength,delta_f1_f2_original_shape_Maximum2DDiameterColumn,delta_f1_f2_original_shape_Maximum2DDiameterRow,delta_f1_f2_original_shape_Maximum2DDiameterSlice,delta_f1_f2_original_shape_Maximum3DDiameter,delta_f1_f2_original_shape_MeshVolume,delta_f1_f2_original_shape_MinorAxisLength,...,delta_f1_f2_original_gldm_LargeDependenceLowGrayLevelEmphasis,delta_f1_f2_original_gldm_LowGrayLevelEmphasis,delta_f1_f2_original_gldm_SmallDependenceEmphasis,delta_f1_f2_original_gldm_SmallDependenceHighGrayLevelEmphasis,delta_f1_f2_original_gldm_SmallDependenceLowGrayLevelEmphasis,delta_f1_f2_original_ngtdm_Busyness,delta_f1_f2_original_ngtdm_Coarseness,delta_f1_f2_original_ngtdm_Complexity,delta_f1_f2_original_ngtdm_Contrast,delta_f1_f2_original_ngtdm_Strength
Patient 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Patient 82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Patient 85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Simu/F2


In [ ]:
new_feature_list = ['delta_simu_f2_' + x for x in features_list]
print(new_feature_list)

['delta_simu_f2_original_shape_Elongation', 'delta_simu_f2_original_shape_Flatness', 'delta_simu_f2_original_shape_LeastAxisLength', 'delta_simu_f2_original_shape_MajorAxisLength', 'delta_simu_f2_original_shape_Maximum2DDiameterColumn', 'delta_simu_f2_original_shape_Maximum2DDiameterRow', 'delta_simu_f2_original_shape_Maximum2DDiameterSlice', 'delta_simu_f2_original_shape_Maximum3DDiameter', 'delta_simu_f2_original_shape_MeshVolume', 'delta_simu_f2_original_shape_MinorAxisLength', 'delta_simu_f2_original_shape_Sphericity', 'delta_simu_f2_original_shape_SurfaceArea', 'delta_simu_f2_original_shape_SurfaceVolumeRatio', 'delta_simu_f2_original_shape_VoxelVolume', 'delta_simu_f2_original_firstorder_10Percentile', 'delta_simu_f2_original_firstorder_90Percentile', 'delta_simu_f2_original_firstorder_Energy', 'delta_simu_f2_original_firstorder_Entropy', 'delta_simu_f2_original_firstorder_InterquartileRange', 'delta_simu_f2_original_firstorder_Kurtosis', 'delta_simu_f2_original_firstorder_Maximu

In [ ]:
data, error_patients = get_data_from_csv('_IRM_simu_mridian_gtv,nii', '_mridian_ttt_2_gtv,nii', '_IRM_simu_MRIdian_gtv,nii')
# Créer le DataFrame
delta_simu_f2_df = pd.DataFrame(data, index=patients_list, columns=new_feature_list)
print("Patient indexes with errors: ", error_patients)

Patient indexes with errors:  ['04', '05', '06', '07', '17', '18', '19', '21', '22', '23', '24', '26', '32', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86']


### Simu/F3 

In [ ]:
new_feature_list = ['delta_simu_f3_' + x for x in features_list]
print(new_feature_list)

['delta_simu_f3_original_shape_Elongation', 'delta_simu_f3_original_shape_Flatness', 'delta_simu_f3_original_shape_LeastAxisLength', 'delta_simu_f3_original_shape_MajorAxisLength', 'delta_simu_f3_original_shape_Maximum2DDiameterColumn', 'delta_simu_f3_original_shape_Maximum2DDiameterRow', 'delta_simu_f3_original_shape_Maximum2DDiameterSlice', 'delta_simu_f3_original_shape_Maximum3DDiameter', 'delta_simu_f3_original_shape_MeshVolume', 'delta_simu_f3_original_shape_MinorAxisLength', 'delta_simu_f3_original_shape_Sphericity', 'delta_simu_f3_original_shape_SurfaceArea', 'delta_simu_f3_original_shape_SurfaceVolumeRatio', 'delta_simu_f3_original_shape_VoxelVolume', 'delta_simu_f3_original_firstorder_10Percentile', 'delta_simu_f3_original_firstorder_90Percentile', 'delta_simu_f3_original_firstorder_Energy', 'delta_simu_f3_original_firstorder_Entropy', 'delta_simu_f3_original_firstorder_InterquartileRange', 'delta_simu_f3_original_firstorder_Kurtosis', 'delta_simu_f3_original_firstorder_Maximu

In [ ]:
data, error_patients = get_data_from_csv('_IRM_simu_mridian_gtv,nii', '_mridian_ttt_3_gtv,nii', '_IRM_simu_MRIdian_gtv,nii')
# Créer le DataFrame
delta_simu_f3_df = pd.DataFrame(data, index=patients_list, columns=new_feature_list)
print("Patient indexes with errors: ", error_patients)

Patient indexes with errors:  ['04', '05', '06', '07', '17', '18', '19', '21', '22', '23', '24', '26', '32', '38', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86']


## Do the same for F1/F2, F1/F3, Simu/F2, Simu/F3 

## Do the same for PTV mask